In [1]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import h5py

In [2]:
def read_daymet_h5(filename):
    data = {}
    with h5py.File(filename, 'r') as f:
        for k, v in f.items():
            try:
                data[k] = v[:]
            except TypeError:
                data_t = {}
                for tk, tv in v.items():
                    data_t[tk] = tv[:]
                data[k] = data_t
    return data

In [3]:
def write_daymet_h5(filename, data):
    with h5py.File(filename, 'w') as f:
        for k, v in data.items():
            try:
                f.create_dataset(k, data=v)
            except TypeError:
                g = f.create_group(k)
                for tk, tv in v.items():
                    g.create_dataset(tk, data=tv)

In [4]:
def check_keys(data):
    keys = data.keys()
    for key in keys:
        print(key, type(data[key]))
        if isinstance(data[key], dict):
            _keys = data[key].keys()
            last = np.array(sorted(np.array(list(_keys), dtype=int)))[-1]
            print('\t', last, ':', data[key][str(last)].shape, type(data[key][str(last)]))
    return last

In [5]:
def _last(data):
    return np.array(sorted(np.array(list(data[list(data.keys())[0]].keys()), dtype=int)))[-1]

In [6]:
domain = 'Yakima'

In [7]:
years = np.arange(2024, 2025)

In [8]:
month = 10 # 12

In [9]:
number_of_days = {1:31, 2:28, 3:31, 4:30, 5:31, 6:30, 7:31, 8:31, 9:30, 10:31, 11:30, 12:31}

In [10]:
flist = [f'./h5/hrrr_{year}{str(month).zfill(2)}_{number_of_days[month]}d.h5' for year in years]
flist

['./h5/hrrr_202410_31d.h5']

In [11]:
data = read_daymet_h5(flist[0])
last = check_keys(data)
last

air temperature [K] <class 'dict'>
	 7295 : (71, 58) <class 'numpy.ndarray'>
incoming shortwave radiation [W m^-2] <class 'dict'>
	 7295 : (71, 58) <class 'numpy.ndarray'>
precipitation rain [m s^-1] <class 'dict'>
	 7295 : (71, 58) <class 'numpy.ndarray'>
precipitation snow [m SWE s^-1] <class 'dict'>
	 7295 : (71, 58) <class 'numpy.ndarray'>
vapor pressure air [Pa] <class 'dict'>
	 7295 : (71, 58) <class 'numpy.ndarray'>


7295

In [12]:
for f in flist[1:]:
    last = _last(data)
    _data = read_daymet_h5(f)
    for key in data.keys():
        for _key in _data[key].keys():
            data[key][str(int(_key)+last+1)] = _data[key][_key]
    print(f, _last(data))

In [13]:
last = check_keys(data)
last

air temperature [K] <class 'dict'>
	 7295 : (71, 58) <class 'numpy.ndarray'>
incoming shortwave radiation [W m^-2] <class 'dict'>
	 7295 : (71, 58) <class 'numpy.ndarray'>
precipitation rain [m s^-1] <class 'dict'>
	 7295 : (71, 58) <class 'numpy.ndarray'>
precipitation snow [m SWE s^-1] <class 'dict'>
	 7295 : (71, 58) <class 'numpy.ndarray'>
vapor pressure air [Pa] <class 'dict'>
	 7295 : (71, 58) <class 'numpy.ndarray'>


7295

In [14]:
raster_shape = (71, 58) # yrb (71, 58) # mkz (29, 40)
xyz_daymet = np.loadtxt('./xyz_csv/yrb/xyz_daymet.xyz')
x_daymet, y_daymet = xyz_daymet[:, 0].reshape(raster_shape), xyz_daymet[:, 1].reshape(raster_shape)
xv, yv = np.meshgrid(np.linspace(x_daymet.min(), x_daymet.max(), raster_shape[1]), np.linspace(y_daymet.min(), y_daymet.max(), raster_shape[0]))
data["x [m]"], data["y [m]"] = xv[0, :], yv[:, 0]

In [15]:
(years[-1]-years[0]+1)*365*24

8760

In [16]:
data["time [s]"] = np.arange((years[-1]-years[0]+1)*365*24)*3600 + (years[0]-1980)*365*86400

In [17]:
write_daymet_h5(f'./{domain}_HRRR_{years[0]}_{years[-1]}.h5', data)